<a href="https://colab.research.google.com/github/arijitaurko/PDF-QA-ChatBot_Llama2-Llamaindex/blob/main/PDF_q_a_notebook_using_Llama2_AstraDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chat with your PDF files using LlamaIndex, Astra DB (Apache Cassandra), and Gradient's open-source models, including LLama2 and Streamlit, all designed for seamless interaction with PDF files.

# Installation

In [ ]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index-core
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q llama-index-llms-gradient
!pip install -q llama-index-embeddings-gradient
!pip install -q llama-index-vector-stores-cassandra
!pip install -q llama-index-readers-file

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 37.7 MB/s eta 0:00:00


# Import OS & JSON Modules

In [ ]:
import os
import json
from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = '49SxShZ6rRvQu8YVAeSjoj3a91rqBWZm'
os.environ['GRADIENT_WORKSPACE_ID'] =  '6114a445-d716-4ff4-ac7b-a7ab9ad42995_workspace'

# Import Cassandra & llama Index

In [ ]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index.core import ServiceContext
from llama_index.core import set_global_service_context
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings.gradient import GradientEmbedding
from llama_index.llms.gradient import GradientBaseModelLLM, GradientModelAdapterLLM
from llama_index.vector_stores.cassandra import CassandraVectorStore

In [ ]:
import cassandra
print (cassandra.__version__)

3.29.0


# Connect to the VectorDB

In [ ]:
!gdown 1KXYTpt7_pTPX4fXh4IQZqbsmOKSTnZOi

Downloading...
From: https://drive.google.com/uc?id=1KXYTpt7_pTPX4fXh4IQZqbsmOKSTnZOi
To: /content/secure-connect-pdf-summarization.zip
100% 12.3k/12.3k [00:00<00:00, 33.8MB/s]


In [ ]:
!gdown 1_3PebirQmKMQITtEYQKa-O6CeuCOhXwx

Downloading...
From: https://drive.google.com/uc?id=1_3PebirQmKMQITtEYQKa-O6CeuCOhXwx
To: /content/pdf-summarization-token.json.txt
100% 496/496 [00:00<00:00, 2.45MB/s]


In [ ]:
# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-pdf-summarization.zip'
}

# if yours is different update the file name below
with open("pdf-summarization-token.json") as f:
    secrets = json.load(f)
CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
# This token json file is autogenerated when you donwload your token,
if row:
  print(row[0])
else:

  print("An error occurred.")

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(138630109961696) 7f83923a-fcee-4bc3-bda5-920b9dde9b08-us-east-1.db.astra.datastax.com:29042:505d2fbb-5c66-4a77-a32b-cf3056bded3e> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


4.0.11-be972d70632f


# Define the Gradient's Model Adapter for LLAMA-2

In [ ]:
from gradientai import Gradient

In [ ]:
#llm = GradientBaseModelLLM(
#    base_model_slug="llama2-7b-chat",
#    max_tokens=400,
#)

gradient = Gradient()
base_model = gradient.get_base_model(base_model_slug = "llama2-7b-chat")

In [ ]:
new_model_adapter = base_model.create_model_adapter(name = 'FineTunedLlama2')

In [ ]:
print(f"Model Adapter Id {new_model_adapter.id}")

Model Adapter Id 30fa2b31-c48e-4744-9328-7e722a7bf9de_model_adapter


In [ ]:
sample_query = "### Instruction: Can you please tell me what a RFP Document is? \n\n Response:"
print(f'Asking: {sample_query}')
completion = new_model_adapter.complete(query = sample_query, max_generated_token_count = 400).generated_output
print(f"Generated before FineTuning: {completion}")

Asking: ### Instruction: Can you please tell me what a RFP Document is? 

 Response:
Generated before FineTuning:  Sure, I'd be happy to explain what a Request for Proposal (RFP) document is!

A Request for Proposal (RFP) is a document that organizations


In [ ]:
variations = [
    {"inputs": "### Instruction: Define an RFP Document \n\n Response: An RFP Document, short for Request for Proposal document, is a formal solicitation that organizations use to invite bids from potential vendors or service providers for a specific project or service. The RFP outlines the project requirements, objectives, and scope, providing detailed information to guide potential respondents in preparing their proposals."},
    {"inputs": "### Instruction: Explain the concept of an RFP Document \n\n Response: An RFP Document, also known as a Request for Proposal document, is a formal invitation from organizations to prospective vendors or service providers to submit proposals for a particular project or service. The RFP specifies the project's requirements, goals, and scope, enabling potential bidders to understand the project's needs and submit relevant proposals."},
    {"inputs": "### Instruction: What exactly is meant by an RFP Document? \n\n Response: An RFP Document, which stands for Request for Proposal document, is a formal solicitation used by organizations to request proposals from potential vendors or service providers for a specific project or service. The RFP outlines the project's objectives, requirements, and scope, providing necessary details to facilitate the submission and evaluation of proposals."},
    {"inputs": "### Instruction: Clarify the purpose of an RFP Document \n\n Response: An RFP Document, or Request for Proposal document, serves as a formal request from organizations to potential vendors or service providers for proposals related to a specific project or service. The RFP outlines the project's objectives, requirements, and scope, guiding vendors in preparing and submitting their proposals."},
    {"inputs": "### Instruction: Please elaborate on the significance of an RFP Document \n\n Response: An RFP Document, commonly known as a Request for Proposal document, holds significant importance in procurement processes, as it serves as a formal invitation from organizations to potential vendors or service providers to submit proposals for a specific project or service. The RFP outlines the project's objectives, requirements, and scope, facilitating the selection of the most suitable vendor or service provider for the project."},
    {"inputs": "### Instruction: Could you shed some light on the function of an RFP Document? \n\n Response: An RFP Document, which stands for Request for Proposal document, plays a crucial role in procurement processes by serving as a formal invitation from organizations to potential vendors or service providers to submit proposals for a specific project or service. The RFP outlines the project's objectives, requirements, and scope, guiding vendors in preparing and submitting their proposals for evaluation and selection."},
    {"inputs": "### Instruction: Kindly provide an overview of an RFP Document \n\n Response: An RFP Document, also referred to as a Request for Proposal document, provides a comprehensive overview of a project or service for which organizations seek proposals from potential vendors or service providers. The RFP outlines the project's objectives, requirements, and scope, facilitating the submission and evaluation of proposals by interested parties."},
    {"inputs": "### Instruction: Shed some light on the purpose of an RFP Document \n\n Response: An RFP Document, short for Request for Proposal document, serves as a formal request from organizations to potential vendors or service providers for proposals related to a specific project or service. The RFP outlines the project's objectives, requirements, and scope, providing necessary details to facilitate the submission and evaluation of proposals."},
    {"inputs": "### Instruction: What role does an RFP Document play in procurement processes? \n\n Response: An RFP Document, commonly known as a Request for Proposal document, plays a pivotal role in procurement processes by serving as a formal invitation from organizations to potential vendors or service providers to submit proposals for a specific project or service. The RFP outlines the project's objectives, requirements, and scope, guiding vendors in preparing and submitting their proposals for evaluation and selection."},
    {"inputs": "### Instruction: Can you provide a brief explanation of an RFP Document? \n\n Response: An RFP Document, which stands for Request for Proposal document, is a formal solicitation used by organizations to request proposals from potential vendors or service providers for a specific project or service. The RFP outlines the project's objectives, requirements, and scope, providing necessary details to facilitate the submission and evaluation of proposals."},
]

In [ ]:
num_epochs = 5
count = 0

while count < num_epochs:
  print(f"Fine-tuning the model, Epoch iteration => {count + 1}")
  new_model_adapter.fine_tune(samples = variations)
  count += 1

Fine-tuning the model, Epoch iteration => 1
Fine-tuning the model, Epoch iteration => 2
Fine-tuning the model, Epoch iteration => 3
Fine-tuning the model, Epoch iteration => 4
Fine-tuning the model, Epoch iteration => 5


In [ ]:
# After fine-tuning
completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
print(f"After Fine-Tuning: {completion}")

After Fine-Tuning:  An RFP Document, which stands for Request for Proposal document, is a formal invitation from organizations to potential vendors or service providers to submit proposals for a specific project or service. The RFP outlines the project's requirements, objectives, and scope, guiding vendors in preparing and submitting their proposals for evaluation and selection. The RFP serves as a formal invitation to tender, facilitating the procurement process by providing necessary details


# Configure Gradient embeddings

In [ ]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

In [ ]:
llm = GradientModelAdapterLLM(
    model_adapter_id = "30fa2b31-c48e-4744-9328-7e722a7bf9de_model_adapter",
    max_tokens = 500)

In [ ]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

<ipython-input-39-944266470549>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


# Load the PDFs

In [ ]:
!gdown 1hkk7-Xg8AOqRZbb4v4nfkcq6OtqRYsAr

Downloading...
From: https://drive.google.com/uc?id=1hkk7-Xg8AOqRZbb4v4nfkcq6OtqRYsAr
To: /content/RFP 972-12-11-2021 Software.pdf
100% 344k/344k [00:00<00:00, 28.8MB/s]


In [ ]:
!mkdir /content/Documents/

In [ ]:
!mv '/content/RFP 972-12-11-2021 Software.pdf' '/content/Documents/RFP 972-12-11-2021 Software.pdf'

In [ ]:
documents = SimpleDirectoryReader("/content/Documents").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 20 document(s).


# Setup and Query Index

In [ ]:
index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What is the deadline of submission?")
print(response)

 Based on the additional context provided, the deadline for submission is still November 05, 2021, during the CSIR's business hours (08h00 - 16h30). However, it is important to note that any tender submitted after the stipulated deadline and place will be regarded as a late tender and will not be considered. Therefore, it is crucial to ensure that the tender is submitted on time and at the correct location to avoid any issues.

The original answer remains unchanged:
The deadline for submission is November 05, 2021.


In [ ]:
response_2 = query_engine.query("What are the in-scopes of Hosted Server?")
print(response_2)

 Based on the new context provided, the in-scopes of Hosted Server can be further refined as follows:

* Transfer the IB Portal and CPMS from the current hosting platform to the new hosting platform.
* Ongoing hosting and upgrade of the platform systems, including security kernel patches, server software, and email packages.
* Provide unlimited email packages and monitor mailbox sizes.
* Ensure SSL certificates and general server security.
* Offer ongoing bug fixes and small changes, limited by the service level agreement and associated budget.
* Maintain ownership of the three domains currently active for the Innovation Bridge Portal and purchase one domain for the CPMS.

Therefore, the Hosted Server is in-scope for transferring the existing IB Portal and CPMS to a new hosting platform, as well as providing ongoing hosting and upgrading of the platform systems, email services, and domain management.


In [ ]:
response_3 = query_engine.query("What is the Average Score Rating of Responsive Design and Development for Mobile Phones, Tablets and PC?")
print(response_3)

 Based on the updated information provided in the RFP document, the Average Score Rating of Responsive Design and Development for Mobile Phones, Tablets, and PC is refined to 6.

The refined rating is based on the following criteria and their corresponding scores:

* No response = 0
* Poor = 1
* Average = 6
* Good = 8
* Very good = 10
* Excellent = 10

Therefore, the average score rating for Responsive Design and Development is 6, which falls under the "Good" category.

The refined answer takes into account the additional information provided in the RFP document regarding the experience required for the development team. With at least 3-4 years of experience in multi-device, multi-browser development and at least 2 examples of relevant systems designed, the team's score has been upgraded to "Good".


For Streamlit application, please refer the following blog: https://www.analyticsvidhya.com/blog/2024/01/automated-fine-tuning-of-llama2-models-on-gradient-ai-cloud/